#### IMPORTACION DE PAQUETES Y DATOS

In [32]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

df = pd.read_csv('train.csv')
df.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [33]:
X = df[[u'Pclass', u'Sex', u'Age', u'SibSp', u'Parch', u'Fare', u'Embarked']]
y = df[u'Survived']

#### IMPUTACION DE CASOS PERDIDOS CATEGORICOS

In [38]:
for i in X.columns.values:
    print i,":", sum(X[i].isnull())

Pclass : 0
Sex : 0
Age : 177
SibSp : 0
Parch : 0
Fare : 0
Embarked : 2


In [39]:
s = X['Embarked'].fillna('Q')
X.loc[:,'Embarked'] = s

#### PREPROCESAMIENTO Y PIPELINE

In [40]:
from sklearn.preprocessing import StandardScaler, LabelEncoder,Imputer, OneHotEncoder, MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from IPython.core.debugger import Tracer

#### GENERACION DEL EXTRACTOR DE COLUMNAS

In [41]:
class DataFrameColumnExtracter(TransformerMixin):

    def __init__(self, columns, y = None):
        self.columns = columns

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        col_list = []
        for c in self.columns:
            col_list.append(c)
        return (X[col_list])
        #return np.array(x)

#### PIPELINE

In [43]:
ndebug_here = Tracer()
pipeline = Pipeline([
    ('union', FeatureUnion([
        ('cont', Pipeline([
            ('extract', DataFrameColumnExtracter([u'Age',u'Fare',u'Parch',u'SibSp'])),
            ('impute', Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
            ('scale', StandardScaler())
        ])),
        ('cat', Pipeline([
            ('extract', DataFrameColumnExtracter([u'Pclass', u'Sex',u'Embarked'])),
            
            ('categ', LabelEncoder()), ## # EL PROBLEMA ESTA EN ESTA LINEA
            
            ('onehot', OneHotEncoder(handle_unknown='error', n_values='auto', sparse=False)),
        ])),
    ])
)])

In [44]:
pipeline.fit_transform(X)

TypeError: fit_transform() takes exactly 2 arguments (3 given)